In [1]:
#需要计算出每个时刻需求用电D以及风速分布参数c和kappa下的最优发电方案以及最优的发电成本 以计算动态的regret
#在确定当前时刻D以及风速分布参数c和kappa下的情况下，跑20000次的迭代将结果作为最优发电方案和最优发电成本

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime
from scipy.special import gamma, factorial
import scipy.special as sc

In [3]:
def attack_free_decentralized_economic_dispatch(d_power_demand,c_wind_speed,kappa_wind_speed):
    thermal_power_station_strategy=np.zeros(shape=(4,20000)) 
    wind_power_station_strategy=np.zeros(shape=(1,20000))
    thermal_power_station_dual_variable=np.zeros(shape=(4,20000))
    wind_power_station_dual_variable=np.zeros(shape=(1,20000))
    E=np.zeros(shape=(5,5))
    thermal_power_station_cost=np.zeros(shape=(1,20000)) #火力发电成本
    wind_power_station_cost=np.zeros(shape=(1,20000)) #风力发电成本
    sum_cost=np.zeros(shape=(1,20000)) #总发电成本
    thermal_power_station_eta=[0.0675,0.0675,0.0925,0.0625]
    #thermal_power_station_eta=[0.0375,0.0375,0.0625,0.035]
    thermal_power_station_zeta=[2,1.75,1,3]
    thermal_power_station_xi=[0,0,0,0]
    thermal_power_station_P_min=[50,20,15,10]
    thermal_power_station_P_max=[200,120,80,100]
    wind_power_station_varphi=[1]
    wind_power_station_v_in=[3]
    wind_power_station_v_out=[25]
    wind_power_station_v_r=[13]
    wind_power_station_sigma_ue=[5]
    wind_power_station_sigma_oe=[30]
    wind_power_station_P_r=[160]
    wind_power_station_P_min=[0]
    wind_power_station_P_max=[160]
    sum_thermal_wind_power_station_strategy=np.zeros(shape=(1,20000))
    step_size_primal=1
    step_size_dual=0.1
    for i in range(0,5):
        for j in range(0,5):
            E[i][j]=1/5
    
    #火力，风力发电策略初始化
    thermal_power_station_strategy[0][0]=0
    thermal_power_station_strategy[1][0]=0
    thermal_power_station_strategy[2][0]=0
    thermal_power_station_strategy[3][0]=0
    wind_power_station_strategy[0][0]=0
    
    #开始迭代
    for k in range(1,20000):
        #发电站更新计划发电量和对偶变量
        #对偶变量 先做梯度上升 再做聚合
        #梯度上升
        dual_variable_intermediate=[] #存放当前迭代中6个用户对对偶变量做完梯度下降后得到的中间值
        for i in range(0,4):
            dual_variable_intermediate.append(thermal_power_station_dual_variable[i][k-1]+step_size_dual*((1/5)*thermal_power_station_strategy[i][k-1]-(1/5)*d_power_demand))
        for j in range(0,1):
            dual_variable_intermediate.append(wind_power_station_dual_variable[j][k-1]+step_size_dual*((1/5)*wind_power_station_strategy[j][k-1]-(1/5)*d_power_demand))
        #聚合 火力发电站的对偶更新
        for i in range(0,4):
            weight_aggregation_dual_variable=0
            for j in range(0,4):
                weight_aggregation_dual_variable=weight_aggregation_dual_variable+E[i][j]*dual_variable_intermediate[j]               
            for j in range(0,1):
                weight_aggregation_dual_variable=weight_aggregation_dual_variable+E[i][j+4]*dual_variable_intermediate[j+4]
            thermal_power_station_dual_variable[i][k]=weight_aggregation_dual_variable
        #聚合 风力发电站的对偶更新
        for i in range(0,1):
            weight_aggregation_dual_variable=0
            for j in range(0,4):
                weight_aggregation_dual_variable=weight_aggregation_dual_variable+E[i+4][j]*dual_variable_intermediate[j]               
            for j in range(0,1):
                weight_aggregation_dual_variable=weight_aggregation_dual_variable+E[i+4][j+4]*dual_variable_intermediate[j+4]
            wind_power_station_dual_variable[i][k]=weight_aggregation_dual_variable              
        #火力发电站计划发电量
        for i in range(0,4):
            thermal_power_station_strategy[i][k]=thermal_power_station_strategy[i][k-1]-step_size_primal*(2*thermal_power_station_eta[i]*thermal_power_station_strategy[i][k-1]+thermal_power_station_zeta[i]+(1/5)*thermal_power_station_dual_variable[i][k-1]) 
            if thermal_power_station_strategy[i][k]<=thermal_power_station_P_min[i]:
                thermal_power_station_strategy[i][k]=thermal_power_station_P_min[i]
            elif thermal_power_station_strategy[i][k]>=thermal_power_station_P_max[i]:
                thermal_power_station_strategy[i][k]=thermal_power_station_P_max[i]
        #风力发电站计划发电量
        for i in range(0,1):
            wind_power_station_strategy[i][k]=wind_power_station_strategy[i][k-1]-step_size_primal*(wind_power_station_varphi[i]+wind_power_station_sigma_oe[i]+(wind_power_station_sigma_ue[i]+wind_power_station_sigma_oe[i])*math.exp(-(math.pow(wind_power_station_v_out[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed)))+(-wind_power_station_sigma_ue[i]-wind_power_station_sigma_oe[i])*math.exp(-(math.pow((wind_power_station_v_in[i]+((wind_power_station_v_r[i]-wind_power_station_v_in[i])*wind_power_station_strategy[i][k-1])/wind_power_station_P_r[i]),kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed)))+(1/5)*wind_power_station_dual_variable[i][k-1])
            if wind_power_station_strategy[i][k]<=wind_power_station_P_min[i]:
                wind_power_station_strategy[i][k]=wind_power_station_P_min[i]
            elif wind_power_station_strategy[i][k]>=wind_power_station_P_max[i]:
                wind_power_station_strategy[i][k]=wind_power_station_P_max[i]
        
        #计算总计划发电
        #火力发电量
        for i in range(0,4):
            sum_thermal_wind_power_station_strategy[0][k]=sum_thermal_wind_power_station_strategy[0][k]+thermal_power_station_strategy[i][k]
        #风力发电量
        for i in range(0,1):
            sum_thermal_wind_power_station_strategy[0][k]=sum_thermal_wind_power_station_strategy[0][k]+wind_power_station_strategy[i][k]
        
        #计算总发电成本
        #火力发电成本
        for i in range(0,4):
            thermal_power_station_cost[0][k]=thermal_power_station_cost[0][k]+thermal_power_station_eta[i]*thermal_power_station_strategy[i][k]*thermal_power_station_strategy[i][k]+thermal_power_station_zeta[i]*thermal_power_station_strategy[i][k]+thermal_power_station_xi[i]
        #风力发电成本
        for i in range(0,1):
            wind_power_station_cost[0][k]=wind_power_station_cost[0][k]+wind_power_station_varphi[i]*wind_power_station_strategy[i][k]+wind_power_station_sigma_ue[i]*((wind_power_station_P_r[i]-wind_power_station_strategy[i][k])*(math.exp(-(math.pow(wind_power_station_v_r[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed)))-math.exp(-(math.pow(wind_power_station_v_out[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed))))+(wind_power_station_P_r[i]*wind_power_station_v_in[i]/(wind_power_station_v_r[i]-wind_power_station_v_in[i])+wind_power_station_strategy[i][k])*(math.exp(-(math.pow(wind_power_station_v_r[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed)))-math.exp(-(math.pow(wind_power_station_v_in[i]+((wind_power_station_v_r[i]-wind_power_station_v_in[i])*wind_power_station_strategy[i][k])/wind_power_station_P_r[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed))))+(wind_power_station_P_r[i]*c_wind_speed/(wind_power_station_v_r[i]-wind_power_station_v_in[i]))*(sc.gammaincc(1+1/kappa_wind_speed,(math.pow(wind_power_station_v_in[i]+((wind_power_station_v_r[i]-wind_power_station_v_in[i])*wind_power_station_strategy[i][k])/wind_power_station_P_r[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed)))*sc.gamma(1+1/kappa_wind_speed)-sc.gammaincc(1+1/kappa_wind_speed,math.pow(wind_power_station_v_r[i]/c_wind_speed,kappa_wind_speed))*sc.gamma(1+1/kappa_wind_speed)))+wind_power_station_sigma_oe[i]*((wind_power_station_strategy[i][k])*(1-math.exp(-(math.pow(wind_power_station_v_in[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed)))+math.exp(-(math.pow(wind_power_station_v_out[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed))))+(wind_power_station_P_r[i]*wind_power_station_v_in[i]/(wind_power_station_v_r[i]-wind_power_station_v_in[i])+wind_power_station_strategy[i][k])*(math.exp(-(math.pow(wind_power_station_v_in[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed)))-math.exp(-(math.pow(wind_power_station_v_in[i]+((wind_power_station_v_r[i]-wind_power_station_v_in[i])*wind_power_station_strategy[i][k])/wind_power_station_P_r[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed))))+(wind_power_station_P_r[i]*c_wind_speed/(wind_power_station_v_r[i]-wind_power_station_v_in[i]))*(sc.gammaincc(1+1/kappa_wind_speed,(math.pow(wind_power_station_v_in[i]+((wind_power_station_v_r[i]-wind_power_station_v_in[i])*wind_power_station_strategy[i][k])/wind_power_station_P_r[i],kappa_wind_speed))/(math.pow(c_wind_speed,kappa_wind_speed)))*sc.gamma(1+1/kappa_wind_speed)-sc.gammaincc(1+1/kappa_wind_speed,math.pow(wind_power_station_v_in[i]/c_wind_speed,kappa_wind_speed))*sc.gamma(1+1/kappa_wind_speed)))
        #发电总成本
        sum_cost[0][k]=thermal_power_station_cost[0][k]+wind_power_station_cost[0][k]
        
    #print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    optimal_power_station_strategy_def=[]
    for i in range(0,4):
        optimal_power_station_strategy_def.append(thermal_power_station_strategy[i][19999])
    for i in range(0,1):
        optimal_power_station_strategy_def.append(wind_power_station_strategy[i][19999])
        
    optimal_dual_variable_def=[]
    for i in range(0,4):
        optimal_dual_variable_def.append(thermal_power_station_dual_variable[i][19999])
    for i in range(0,1):
        optimal_dual_variable_def.append(wind_power_station_dual_variable[i][19999])
     
    return sum_cost[0][19999], optimal_power_station_strategy_def, optimal_dual_variable_def

In [4]:
#定义变量 
d_power_demand_list=np.loadtxt("./data/d_power_demand_normal_distribution_(70,5).txt") #时变的需求侧用电需求  
c_wind_speed_list= np.loadtxt("./data/c_wind_speed_uniform_distribution_(3,25).txt")      
kappa_wind_speed_list= np.loadtxt("./data/kappa_wind_speed_uniform_distribution_(2,3).txt") 
optimal_sum_cost=np.zeros(shape=(1,1000)) #最优的发电成本
optimal_power_strategy=np.zeros(shape=(1000,5)) #最优的发电策略
optimal_dual_variable=np.zeros(shape=(1000,5)) #最优的对偶变量

In [5]:
for i in range(0,1000):
    d_power_demand=d_power_demand_list[i]
    c_wind_speed=c_wind_speed_list[i]
    kappa_wind_speed=kappa_wind_speed_list[i]
    optimal_sum_cost[0][i], optimal_power_strategy[i], optimal_dual_variable[i]=attack_free_decentralized_economic_dispatch(d_power_demand,c_wind_speed,kappa_wind_speed)
    if (i+1)%100==0:
        print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])   

2024-03-09 20:55:00.608
2024-03-09 20:57:59.155
2024-03-09 21:00:57.919
2024-03-09 21:03:54.982
2024-03-09 21:06:51.431
2024-03-09 21:09:47.602
2024-03-09 21:12:43.336
2024-03-09 21:15:44.430
2024-03-09 21:18:55.347
2024-03-09 21:22:08.993


In [6]:
#attack_free_decentralized_economic_dispatch(d_power_demand)

In [7]:
np.savetxt('./data/00_optimal_sum_cost_varying_D_c_kappa_normal_distribution_(70,5).txt',optimal_sum_cost,fmt='%0.8f')
np.savetxt('./data/00_optimal_power_strategy_varying_D_c_kappa_normal_distribution_(70,5).txt',optimal_power_strategy,fmt='%0.8f')
np.savetxt('./data/00_optimal_dual_variable_varying_D_c_kappa_normal_distribution_(70,5).txt',optimal_dual_variable,fmt='%0.8f')

In [8]:
optimal_sum_cost

array([[2819.75651114, 2160.70627615, 2619.9235129 , 2842.18989436,
        2349.12076363, 2566.95458685, 2381.75258807, 2539.78363094,
        2587.40059872, 2147.49331697, 3863.58350893, 2813.19887871,
        2212.95491963, 2788.2276884 , 2443.0396152 , 2289.48915089,
        2926.25948258, 2222.98166458, 3385.16987332, 2536.22913607,
        2951.75762014, 2627.52879605, 3065.61174483, 2662.36374292,
        3405.04962718, 1975.7139628 , 2743.03008362, 2959.63910813,
        2604.76957213, 3057.72017459, 2698.57893442, 2721.59215014,
        2556.96720623, 2860.98336065, 2708.4863304 , 2599.71632518,
        2202.68841918, 2498.22340253, 1488.4656643 , 1899.94928885,
        3006.53523264, 2460.25504957, 2441.51967335, 3163.47616715,
        2288.84099896, 2473.27899864, 2263.34555097, 3089.93929682,
        2624.00963622, 2706.06570307, 2964.22662587, 1988.14519501,
        3161.52085801, 2156.82407927, 2840.40143596, 3016.28477339,
        2372.41795491, 2206.81728479, 2408.13167

In [9]:
optimal_power_strategy[0]

array([78.56265198, 80.41450383, 62.7349082 , 76.84766414, 64.57933465])

In [10]:
optimal_dual_variable[0]

array([-63.02979009, -63.02979009, -63.02979009, -63.02979009,
       -63.02979009])